# Semblance Calculation:

This module contains the scripts to perform the semblance calculation for input and output checkerboard. The input checkerboards are created by multipying three sin functions in three directions. The output checkerboards are obtained from the LOTOS code. The semblance is calculated following Kiser et al. 2016 after averaging over neighbouring nodes.The step by step process is shown below:

## 1. Import Modules

In [3]:
import numpy as np
import pygmt
import pandas as pd
from scipy.interpolate import interp1d
from functools import reduce
import matplotlib.pyplot as plt
from math import *

## 2. Run the shell script to obtain the model file

In [31]:
%%sh
home="/Users/sankha/Desktop/research/southern_puna/lotos"

cd $home/PROGRAMS/2_INVERS_3D/8n_3D_model
./mod_3D.exe>$home/checkerboard_test/check20_20_8/dv3D_synth001.dat


cp $home/DATA/PUNASLP1/SYNTH001/data/refmod.dat $home/checkerboard_test/check20_20_8/

## 3. Clean up the input file and refmod file from text before running further codes.

## 4. Interpolate through 1D model and calculate percentage change for each anomaly:

Run the script twice as the file is not written completely on the first run for some reason.

In [6]:
#Add the absolute velocity with the anomaly using refmod.dat

fo=open("./check20_20_8/dv3D_synth001_r1_neu_abs_p.dat",'w') #output file
df_mod=pd.read_csv("./check20_20_8/refmod.dat", sep="\s+",header=None) #reference 1D model file

#Interpolate either P or S velocity
f1=interp1d(df_mod[0],df_mod[1])

#Check Z range in major_param.dat
dzt=np.arange(-4,21,1)

#Interpolate at Z values
dvt=f1(dzt)

#print((dzt,dvt))

df_obt=pd.read_csv("./check20_20_8/dv3D_synth001_r1.dat",sep="\s+",header=None)

#Adds a column of absolute velocity and change in velocity in percent.
k=0
for i in range(len(df_obt[0])):
    
    if df_obt[0][i]==1: #P or S
        print(df_obt[0][i],df_obt[1][i],df_obt[2][i],df_obt[3][i],df_obt[4][i],dvt[k],(df_obt[4][i]/dvt[k])*100,file=fo)
        k=k+1
        if k%25==0: #be careful for each file
            k=0

## 5. Transform nodes to coordinates (optional)

In [8]:
#Nodes to coordinates for velocity model
lat0=-26.7
lon0=-67.1
dep0=-4.0
df_real = pd.read_csv("./check20_20_8/dv3D_synth001_r1_neu_abs_p.dat",sep="\s+",header=None)
#print(df_real)
a1=4.0/110.574
a2=4.0/(111.320*cos(lat0*pi/180))
lonlist=round((lon0+(df_real[1]-36)*a2),4)
latlist=round((lat0+(df_real[2]-36)*a1),4)
deplist=(dep0+df_real[3]-1)

# initialize data of lists.
data = {0: lonlist,1: latlist,2:deplist,3:df_real[4],4:df_real[5],5:df_real[6]}

# Create DataFrame
df = pd.DataFrame(data)
print(df)
df.to_csv("./check20_20_8/dv3D_synth001_r1_neu_abs_coord_p.dat",sep=" ",index=False,header=None)


              0        1     2    3      4    5
0      -68.5077 -27.9661  -4.0  0.0  5.732  0.0
1      -68.5077 -27.9661  -3.0  0.0  5.744  0.0
2      -68.5077 -27.9661  -2.0  0.0  5.756  0.0
3      -68.5077 -27.9661  -1.0  0.0  5.768  0.0
4      -68.5077 -27.9661   0.0  0.0  5.780  0.0
...         ...      ...   ...  ...    ...  ...
126020 -65.6923 -25.4339  16.0  0.0  5.952  0.0
126021 -65.6923 -25.4339  17.0  0.0  5.984  0.0
126022 -65.6923 -25.4339  18.0  0.0  6.016  0.0
126023 -65.6923 -25.4339  19.0  0.0  6.048  0.0
126024 -65.6923 -25.4339  20.0  0.0  6.080  0.0

[126025 rows x 6 columns]


## 6. Generate the input checkerboard pattern:

We need to do this only for P. For S models, the checkerboard remains same.

In [10]:
d2r=pi/180.  #degree to radian
amp=10.0    #Checkerboard anomaly amplitude

#checkerboard size
checksize_hor=20 
checksize_vert=8
#nodespace
nph=4 
npv=1 
#Node limits
horlimit=[-140,140]  
vertlimit=[-4,20]
#Checkerboard limits
checklimit_x=[-120,120]
checklimit_y=[-120,120]
checklimit_z=[-3,19]
#Period of sin function
tph=2.0*checksize_hor/nph
tpv=2.0*checksize_vert/npv
#Number of nodes
hn=1+(horlimit[1]-horlimit[0])/nph  
vn=1+(vertlimit[1]-vertlimit[0])       
x=np.arange(1,hn+1,1)
y=np.arange(1,hn+1,1)
z=np.arange(1,vn+1,1)
#Nodes where no checkerboards are placed
nochkx = [abs(horlimit[0]-checklimit_x[0])/nph,abs(horlimit[1]-checklimit_x[1])/nph]
nochky = [abs(horlimit[0]-checklimit_y[0])/nph,abs(horlimit[1]-checklimit_y[1])/nph]
nochkz = [abs(vertlimit[0]-checklimit_z[0])/npv,abs(vertlimit[1]-checklimit_z[1])/npv]
xmin=(1+nochkx[0])
ymin=(1+nochky[0])
zmin=(1+nochkz[0])
#print(xmin,(hn-nochkx[1]),ymin,(hn-nochky[1]),zmin,tph,tpv)

fo=open("./check20_20_8/dv3D_synth001_input_intm_p.dat",'w')

for i in x:
    for j in y:
        for k in z:
            if i>=xmin and i<=(hn-nochkx[1]) and j>=ymin and j<=(hn-nochky[1]) and k>=zmin:
                dv=(np.sin(2*pi*(i-xmin)/tph)) * (np.sin(2*pi*(j-ymin)/tph)) * (np.sin(2*pi*(k-zmin)/tpv)) *amp
            else:
                dv=0
            print(i,j,k,round(dv,4),file=fo)
            
#Same sorting as the output model
df=pd.read_csv("./check20_20_8/dv3D_synth001_input_intm_p.dat",sep="\s+",header=None)
df2=df.sort_values(by=[0, 1])
print(df2)
df2.to_csv("./check20_20_8/dv3D_synth001_r1_neu_input_p.dat",sep=" ",index=False,header=None)            

           0     1   2    3
0        1.0   1.0   1  0.0
1        1.0   1.0   2  0.0
2        1.0   1.0   3  0.0
3        1.0   1.0   4  0.0
4        1.0   1.0   5  0.0
...      ...   ...  ..  ...
126020  71.0  71.0  21  0.0
126021  71.0  71.0  22  0.0
126022  71.0  71.0  23  0.0
126023  71.0  71.0  24  0.0
126024  71.0  71.0  25  0.0

[126025 rows x 4 columns]


In [12]:
#Nodes to coordinates for semblance
lat0=-26.7
lon0=-67.1
dep0=-4.0

df_sem = pd.read_csv("./check20_20_8/dv3D_synth001_r1_neu_input_p.dat",sep="\s+",header=None)
a1=4.0/110.574
a2=4.0/(111.320*cos(lat0*pi/180))

lonlist=round((lon0+(df_sem[0]-36)*a2),4)
latlist=round((lat0+(df_sem[1]-36)*a1),4)
deplist=(dep0+df_sem[2]-1)

# initialize data of lists.
data = {0: lonlist,1: latlist,2:deplist,3:df_sem[3]}  
# Create DataFrame
df = pd.DataFrame(data)
print(df)
df.to_csv("./check20_20_8/dv3D_synth001_r1_neu_input_coord_p.dat",sep=" ",index=False,header=None)

              0        1     2    3
0      -68.5077 -27.9661  -4.0  0.0
1      -68.5077 -27.9661  -3.0  0.0
2      -68.5077 -27.9661  -2.0  0.0
3      -68.5077 -27.9661  -1.0  0.0
4      -68.5077 -27.9661   0.0  0.0
...         ...      ...   ...  ...
126020 -65.6923 -25.4339  16.0  0.0
126021 -65.6923 -25.4339  17.0  0.0
126022 -65.6923 -25.4339  18.0  0.0
126023 -65.6923 -25.4339  19.0  0.0
126024 -65.6923 -25.4339  20.0  0.0

[126025 rows x 4 columns]


## 7. Calculate the semblance averaging over nearby nodes

In [13]:
#Semblance on the nodes with neighbours
df_in=pd.read_csv("./check20_20_8/dv3D_synth001_r1_neu_input_p.dat",sep="\s+",header=None)
print(df_in.shape)
df_out=pd.read_csv("./check20_20_8/dv3D_synth001_r1_neu_abs_p.dat",sep="\s+",header=None)
print(df_out.shape)

fo=open("./check20_20_8/semblance_synth001_avgnd_r1_neu_p.dat",'w')
sem=[]
for j in range(len(df_in[0])):
    
    #Find distances from all values
    k1=abs(df_in[0]-df_in[0][j])
    k2=abs(df_in[1]-df_in[1][j])
    k3=abs(df_in[2]-df_in[2][j])
    
    idx=reduce(np.intersect1d,(np.where(k1<=1),np.where(k2<=1),np.where(k3<=1 )))
    
    #Anomaly values at those indices
    PT=(df_in[3][idx])
    PR=(df_out[6][idx])

    #Calculate the semblance
    ll=(np.sum((PT+PR)**2))
    dl=(2*np.sum(PT**2+PR**2))    
    if dl==0:
        s=0.5
    else:
        s=ll/ dl  #Semblance formula
    sem.append(s)        
    print(int(df_in[0][j]),int(df_in[1][j]),int(df_in[2][j]),df_in[3][j],df_out[6][j],s)
    print(int(df_in[0][j]),int(df_in[1][j]),int(df_in[2][j]),df_in[3][j],df_out[6][j],s,file=fo)

(126025, 4)
(126025, 7)
1 1 1 0.0 0.0 0.5
1 1 2 0.0 0.0 0.5
1 1 3 0.0 0.0 0.5
1 1 4 0.0 0.0 0.5
1 1 5 0.0 0.0 0.5
1 1 6 0.0 0.0 0.5
1 1 7 0.0 0.0 0.5
1 1 8 0.0 0.0 0.5
1 1 9 0.0 0.0 0.5
1 1 10 0.0 0.0 0.5
1 1 11 0.0 0.0 0.5
1 1 12 0.0 0.0 0.5
1 1 13 0.0 0.0 0.5
1 1 14 0.0 0.0 0.5
1 1 15 0.0 0.0 0.5
1 1 16 0.0 0.0 0.5
1 1 17 0.0 0.0 0.5
1 1 18 0.0 0.0 0.5
1 1 19 0.0 0.0 0.5
1 1 20 0.0 0.0 0.5
1 1 21 0.0 0.0 0.5
1 1 22 0.0 0.0 0.5
1 1 23 0.0 0.0 0.5
1 1 24 0.0 0.0 0.5
1 1 25 0.0 0.0 0.5
1 2 1 0.0 0.0 0.5
1 2 2 0.0 0.0 0.5
1 2 3 0.0 0.0 0.5
1 2 4 0.0 0.0 0.5
1 2 5 0.0 0.0 0.5
1 2 6 0.0 0.0 0.5
1 2 7 0.0 0.0 0.5
1 2 8 0.0 0.0 0.5
1 2 9 0.0 0.0 0.5
1 2 10 0.0 0.0 0.5
1 2 11 0.0 0.0 0.5
1 2 12 0.0 0.0 0.5
1 2 13 0.0 0.0 0.5
1 2 14 0.0 0.0 0.5
1 2 15 0.0 0.0 0.5
1 2 16 0.0 0.0 0.5
1 2 17 0.0 0.0 0.5
1 2 18 0.0 0.0 0.5
1 2 19 0.0 0.0 0.5
1 2 20 0.0 0.0 0.5
1 2 21 0.0 0.0 0.5
1 2 22 0.0 0.0 0.5
1 2 23 0.0 0.0 0.5
1 2 24 0.0 0.0 0.5
1 2 25 0.0 0.0 0.5
1 3 1 0.0 0.0 0.5
1 3 2 0.0 0.0 0.5
1 3 3 0.

1 18 14 0.0 -0.0029748625762711 0.5
1 18 15 0.0 -0.008376849751693 0.5
1 18 16 0.0 -0.0082741025140924 0.5
1 18 17 0.0 0.1017983005067567 0.5
1 18 18 0.0 0.3073922972972973 0.5
1 18 19 0.0 0.0 0.5
1 18 20 0.0 0.0 0.5
1 18 21 0.0 0.0 0.5
1 18 22 0.0 0.0 0.5
1 18 23 0.0 0.0 0.5
1 18 24 0.0 0.0 0.5
1 18 25 0.0 0.0 0.5
1 19 1 0.0 0.0 0.5
1 19 2 0.0 0.0894977078690807 0.5
1 19 3 0.0 0.180174351980542 0.5
1 19 4 0.0 0.2666140950069348 0.5
1 19 5 0.0 0.3411034671280277 0.5
1 19 6 0.0 0.4085734661697248 0.5
1 19 7 0.0 0.4731795239452679 0.5
1 19 8 0.0 0.540541712585034 0.5
1 19 9 0.0 0.5666000005665723 0.5
1 19 10 0.0 0.5381271693091733 0.5
1 19 11 0.0 0.4960501375212224 0.5
1 19 12 0.0 0.4596321227375566 0.5
1 19 13 0.0 0.4237857388355003 0.5
1 19 14 0.0 0.3879798610169491 0.5
1 19 15 0.0 0.3547564841986456 0.5
1 19 16 0.0 0.3246911465614431 0.5
1 19 17 0.0 0.2939681942567568 0.5
1 19 18 0.0 0.4566543918918918 0.5
1 19 19 0.0 0.0 0.5
1 19 20 0.0 0.0 0.5
1 19 21 0.0 0.0 0.5
1 19 22 0.0 0.0 0.5

1 29 14 0.0 -0.2461627152542373 0.5
1 29 15 0.0 -0.2304095773137697 0.5
1 29 16 0.0 0.0 0.5
1 29 17 0.0 0.0 0.5
1 29 18 0.0 0.0 0.5
1 29 19 0.0 0.0 0.5
1 29 20 0.0 0.0 0.5
1 29 21 0.0 0.0 0.5
1 29 22 0.0 0.0 0.5
1 29 23 0.0 0.0 0.5
1 29 24 0.0 0.0 0.5
1 29 25 0.0 0.0 0.5
1 30 1 0.0 0.0 0.5
1 30 2 0.0 -0.1521688344359331 0.5
1 30 3 0.0 -0.2245126025017373 0.5
1 30 4 0.0 -0.3277411338418863 0.5
1 30 5 0.0 -0.44048576816609 0.5
1 30 6 0.0 -0.517443431766055 0.5
1 30 7 0.0 -0.5043500096921323 0.5
1 30 8 0.0 -0.4784114421768707 0.5
1 30 9 0.0 -0.4561557076487252 0.5
1 30 10 0.0 -0.4360532446206115 0.5
1 30 11 0.0 -0.4147970016977929 0.5
1 30 12 0.0 -0.3917624745475113 0.5
1 30 13 0.0 -0.3687539253815715 0.5
1 30 14 0.0 -0.3457713745762711 0.5
1 30 15 0.0 -0.3213282358916479 0.5
1 30 16 0.0 -0.3041613432919955 0.5
1 30 17 0.0 -0.2641332449324324 0.5
1 30 18 0.0 0.0 0.5
1 30 19 0.0 0.0 0.5
1 30 20 0.0 0.0 0.5
1 30 21 0.0 0.0 0.5
1 30 22 0.0 0.0 0.5
1 30 23 0.0 0.0 0.5
1 30 24 0.0 0.0 0.5
1 30

1 42 6 0.0 0.0 0.5
1 42 7 0.0 0.0 0.5
1 42 8 0.0 0.0 0.5
1 42 9 0.0 0.0 0.5
1 42 10 0.0 0.0 0.5
1 42 11 0.0 0.0 0.5
1 42 12 0.0 0.0 0.5
1 42 13 0.0 0.0 0.5
1 42 14 0.0 0.0 0.5
1 42 15 0.0 0.0 0.5
1 42 16 0.0 0.0 0.5
1 42 17 0.0 0.0 0.5
1 42 18 0.0 0.0 0.5
1 42 19 0.0 0.0 0.5
1 42 20 0.0 0.0 0.5
1 42 21 0.0 0.0 0.5
1 42 22 0.0 0.0 0.5
1 42 23 0.0 0.0 0.5
1 42 24 0.0 0.0 0.5
1 42 25 0.0 0.0 0.5
1 43 1 0.0 0.0 0.5
1 43 2 0.0 0.0 0.5
1 43 3 0.0 0.0 0.5
1 43 4 0.0 0.0 0.5
1 43 5 0.0 0.0 0.5
1 43 6 0.0 0.0 0.5
1 43 7 0.0 0.0 0.5
1 43 8 0.0 0.0 0.5
1 43 9 0.0 0.0 0.5
1 43 10 0.0 0.0 0.5
1 43 11 0.0 0.0 0.5
1 43 12 0.0 0.0 0.5
1 43 13 0.0 0.0 0.5
1 43 14 0.0 0.0 0.5
1 43 15 0.0 0.0 0.5
1 43 16 0.0 0.0 0.5
1 43 17 0.0 0.0 0.5
1 43 18 0.0 0.0 0.5
1 43 19 0.0 0.0 0.5
1 43 20 0.0 0.0 0.5
1 43 21 0.0 0.0 0.5
1 43 22 0.0 0.0 0.5
1 43 23 0.0 0.0 0.5
1 43 24 0.0 0.0 0.5
1 43 25 0.0 0.0 0.5
1 44 1 0.0 0.0 0.5
1 44 2 0.0 0.0 0.5
1 44 3 0.0 0.0 0.5
1 44 4 0.0 0.0 0.5
1 44 5 0.0 0.0 0.5
1 44 6 0.0 0.0 0.5

1 60 12 0.0 0.0 0.5
1 60 13 0.0 0.0 0.5
1 60 14 0.0 0.0 0.5
1 60 15 0.0 0.0 0.5
1 60 16 0.0 0.0 0.5
1 60 17 0.0 0.0 0.5
1 60 18 0.0 0.0 0.5
1 60 19 0.0 0.0 0.5
1 60 20 0.0 0.0 0.5
1 60 21 0.0 0.0 0.5
1 60 22 0.0 0.0 0.5
1 60 23 0.0 0.0 0.5
1 60 24 0.0 0.0 0.5
1 60 25 0.0 0.0 0.5
1 61 1 0.0 0.0 0.5
1 61 2 0.0 0.0 0.5
1 61 3 0.0 0.0 0.5
1 61 4 0.0 0.0 0.5
1 61 5 0.0 0.0 0.5
1 61 6 0.0 0.0 0.5
1 61 7 0.0 0.0 0.5
1 61 8 0.0 0.0 0.5
1 61 9 0.0 0.0 0.5
1 61 10 0.0 0.0 0.5
1 61 11 0.0 0.0 0.5
1 61 12 0.0 0.0 0.5
1 61 13 0.0 0.0 0.5
1 61 14 0.0 0.0 0.5
1 61 15 0.0 0.0 0.5
1 61 16 0.0 0.0 0.5
1 61 17 0.0 0.0 0.5
1 61 18 0.0 0.0 0.5
1 61 19 0.0 0.0 0.5
1 61 20 0.0 0.0 0.5
1 61 21 0.0 0.0 0.5
1 61 22 0.0 0.0 0.5
1 61 23 0.0 0.0 0.5
1 61 24 0.0 0.0 0.5
1 61 25 0.0 0.0 0.5
1 62 1 0.0 0.0 0.5
1 62 2 0.0 0.0 0.5
1 62 3 0.0 0.0 0.5
1 62 4 0.0 0.0 0.5
1 62 5 0.0 0.0 0.5
1 62 6 0.0 0.0 0.5
1 62 7 0.0 0.0 0.5
1 62 8 0.0 0.0 0.5
1 62 9 0.0 0.0 0.5
1 62 10 0.0 0.0 0.5
1 62 11 0.0 0.0 0.5
1 62 12 0.0 0.0 0.

...

7 21 20 0.0 0.0 0.5037355094701708
7 21 21 0.0 0.0 0.5
7 21 22 0.0 0.0 0.5
7 21 23 0.0 0.0 0.5
7 21 24 0.0 0.0 0.5
7 21 25 0.0 0.0 0.5
7 22 1 0.0 0.0 0.5
7 22 2 -0.0 -0.0524559009401114 0.4632409386407219
7 22 3 -1.3221 -0.1035505469075746 0.46910727555753895
7 22 4 -2.443 -0.1608582838072122 0.4698261886416278
7 22 5 -3.1919 -0.2188986539792387 0.4684386573715149
7 22 6 -3.4549 -0.2753513514908257 0.4653090580987086
7 22 7 -3.1919 -0.333486968072976 0.46081519959890654
7 22 8 -2.443 -0.4040427363945578 0.4574922797649228
7 22 9 -1.3221 -0.4958693388101983 0.4578502022463718
7 22 10 -0.0 -0.5123782100792752 0.4889850390957664
7 22 11 1.3221 -0.5004944906621392 0.5100795865839846
7 22 12 2.443 -0.4595697641402715 0.5083895711332598
7 22 13 3.1919 -0.4172545828151498 0.5064484226151387
7 22 14 3.4549 -0.3738109084745762 0.5052064769875176
7 22 15 3.1919 -0.3439572612866817 0.5045755162117651
7 22 16 2.443 -0.3112296409244645 0.5041871175290662
7 22 17 1.3221 -0.280051027027027 0.50424707

7 28 23 5.1646 0.0 0.5071464350879357
7 28 24 3.9528 0.0 0.5
7 28 25 2.1393 0.0 0.5
7 29 1 0.0 0.0 0.5
7 29 2 0.0 0.3233455675487465 0.7282823254635417
7 29 3 2.1393 0.7167598332175121 0.7116247393398634
7 29 4 3.9528 1.133519422676838 0.7099824831022931
7 29 5 5.1646 1.4603003910034602 0.7154718728887323
7 29 6 5.5902 1.7117211227064222 0.7336653620727429
7 29 7 5.1646 1.8820446978335232 0.7627607456152006
7 29 8 3.9528 1.810397193877551 0.8037761587463516
7 29 9 2.1393 1.671703104815864 0.8162744131465077
7 29 10 0.0 1.5901569546998866 0.523721619367394
7 29 11 -2.1393 1.5370412427843805 0.22993130022357314
7 29 12 -3.9528 1.4809045384615385 0.2552810521358967
7 29 13 -5.1646 1.390223755794234 0.2976851269560953
7 29 14 -5.5902 1.2902410355932203 0.3200563584408968
7 29 15 -5.1646 1.1814989441309254 0.3236747569824509
7 29 16 -3.9528 1.0730020417136417 0.30883436137194165
7 29 17 -2.1393 0.9656713243243242 0.28875839134188447
7 29 18 -0.0 0.8570792381756757 0.46701100515295324
7 29 1

7 35 24 -2.443 0.0 0.5091345747276238
7 35 25 -1.3221 0.0 0.5054158057603798
7 36 1 0.0 0.0 0.5
7 36 2 -0.0 0.3527092409470752 0.5506055001247601
7 36 3 -0.0 0.6923884346768588 0.5511639681664161
7 36 4 -0.0 1.027671834257975 0.5588803653150903
7 36 5 -0.0 1.3337177335640138 0.568885702687564
7 36 6 -0.0 1.6345988979357795 0.5862048840639789
7 36 7 -0.0 1.7779063511972637 0.605972377182889
7 36 8 -0.0 1.7016918197278914 0.6272433135947962
7 36 9 -0.0 1.589871161473088 0.6226353008750607
7 36 10 -0.0 1.4693695855039637 0.5079798199711598
7 36 11 0.0 1.3748353989813242 0.38452366783901554
7 36 12 0.0 1.2753139004524887 0.3797776717231105
7 36 13 0.0 1.1764688292820802 0.39593351682170225
7 36 14 0.0 1.0776903677966103 0.4055767029823107
7 36 15 0.0 0.9820932782167042 0.40684069030275954
7 36 16 0.0 0.8868047891770012 0.3992990521585535
7 36 17 0.0 0.7332767668918919 0.38975917792438125
7 36 18 0.0 0.5075298378378379 0.4560513378800005
7 36 19 -0.0 0.4016622466216216 0.5534507407360244
7 

7 44 20 -3.9528 0.0 0.5291102727541871
7 44 21 -5.1646 0.0 0.5197518322670116
7 44 22 -5.5902 0.0 0.5146045710505799
7 44 23 -5.1646 0.0 0.5096651946821767
7 44 24 -3.9528 0.0 0.5061267092856045
7 44 25 -2.1393 0.0 0.5
7 45 1 0.0 0.0 0.5
7 45 2 -0.0 0.0138026050835654 0.5403790439246094
7 45 3 -1.3221 0.0283777070882557 0.5345202570219353
7 45 4 -2.443 0.0425414081137309 0.5332514782633443
7 45 5 -3.1919 0.0640281852941176 0.5348417697502751
7 45 6 -3.4549 0.0773400144495412 0.5394327045043598
7 45 7 -3.1919 0.0846600494298745 0.5473894717411361
7 45 8 -2.443 0.0954595897959183 0.5593786764101706
7 45 9 -1.3221 0.1155616709915014 0.5729969187388627
7 45 10 -0.0 0.0903056881087202 0.5009408244471314
7 45 11 1.3221 0.059166986417657 0.4345088048181904
7 45 12 2.443 0.051718918438914 0.4484713731676449
7 45 13 3.1919 0.0496161008479366 0.4588089837421742
7 45 14 3.4549 0.0449819411864406 0.463161213704009
7 45 15 3.1919 0.0389453321106094 0.4631109616725071
7 45 16 2.443 0.035187720067643

KeyboardInterrupt: 

## 8. Calculate average semblance of checkerboards shifted in 3D

In [164]:
df_1=pd.read_csv("./check20_20_8/semblance_synth001_avgnd_r1_neu_p.dat",sep="\s+",header=None)
df_2=pd.read_csv("./check20_20_8/semblance_synth001_avgnd_r2_xpos_p.dat",sep="\s+",header=None)
df_3=pd.read_csv("./check20_20_8/semblance_synth001_avgnd_r3_xneg_p.dat",sep="\s+",header=None)
df_4=pd.read_csv("./check20_20_8/semblance_synth001_avgnd_r4_ypos_p.dat",sep="\s+",header=None)
df_5=pd.read_csv("./check20_20_8/semblance_synth001_avgnd_r5_yneg_p.dat",sep="\s+",header=None)
df_6=pd.read_csv("./check20_20_8/semblance_synth001_avgnd_r6_zdown_p.dat",sep="\s+",header=None)
df_7=pd.read_csv("./check20_20_8/semblance_synth001_avgnd_r7_zup_p.dat",sep="\s+",header=None)

avgsem=(df_1[5]+df_2[5]+df_3[5]+df_4[5]+df_5[5]+df_6[5]+df_7[5])/7.0
print(avgsem)
df_comb = df_1[[0,1,2,3]].copy()
df_comb[4]=avgsem
df_comb.to_csv("./check20_20_8/semblance_avgnd_average_chk20_p.dat",sep=" ",index=False, header=False)

0         0.5
1         0.5
2         0.5
3         0.5
4         0.5
         ... 
126020    0.5
126021    0.5
126022    0.5
126023    0.5
126024    0.5
Name: 5, Length: 126025, dtype: float64


In [166]:
fo=open("./check20_20_8/semblance_avgnd_average_chk20_comb.dat",'w')
df_1=pd.read_csv("./check20_20_8/semblance_avgnd_average_chk20_p.dat",sep="\s+",header=None)
df_2=pd.read_csv("./check20_20_8/semblance_avgnd_average_chk20_s.dat",sep="\s+",header=None)
#print(df_1)
#print(df_2)
for i in range(len(df_1[0])):
    print(df_1[0][i],df_1[1][i],df_1[2][i],min(df_1[4][i],df_2[4][i]),file=fo)

## 9. Nodes to coordinate transformation for semblance:

In [168]:
#Nodes to coordinates for semblance
lat0=-26.7
lon0=-67.1
dep0=-4.0

df_sem = pd.read_csv("./check20_20_8/semblance_avgnd_average_chk20_comb.dat",sep="\s+",header=None)
a1=4.0/110.574
a2=4.0/(111.320*cos(lat0*pi/180))

lonlist=round((lon0+(df_sem[0]-36)*a2),4)
latlist=round((lat0+(df_sem[1]-36)*a1),4)
deplist=(dep0+df_sem[2]-1)

# initialize data of lists.
data = {0: lonlist,1: latlist,2:deplist,3:df_sem[3]}  
# Create DataFrame
df = pd.DataFrame(data)
print(df)
df.to_csv("./check20_20_8/semblance_average_chk20_coord_comb.dat",sep=" ",index=False,header=None)

              0        1     2    3
0      -68.5077 -27.9661  -4.0  0.5
1      -68.5077 -27.9661  -3.0  0.5
2      -68.5077 -27.9661  -2.0  0.5
3      -68.5077 -27.9661  -1.0  0.5
4      -68.5077 -27.9661   0.0  0.5
...         ...      ...   ...  ...
126020 -65.6923 -25.4339  16.0  0.5
126021 -65.6923 -25.4339  17.0  0.5
126022 -65.6923 -25.4339  18.0  0.5
126023 -65.6923 -25.4339  19.0  0.5
126024 -65.6923 -25.4339  20.0  0.5

[126025 rows x 4 columns]
